In [78]:
## we define the product type here, Landsat 5:"CU_LT05.001", Landsat 7 :"CU_LE07.001", Landsat 8 :"CU_LC08.001"
prods = ["CU_LE07.001"]
layers = [(prods[0], "PIXELQA"), (prods[0], "SRB1"), (prods[0], "SRB2"), (prods[0], "SRB3"), (prods[0], "SRB4"), (prods[0], "SRB5"), (prods[0], "SRB7")]
prodLayer = []
for l in layers:
    prodLayer.append({
        "layer": l[1],
        "product": l[0]
    })
prodLayer

[{'layer': 'PIXELQA', 'product': 'CU_LE07.001'},
 {'layer': 'SRB1', 'product': 'CU_LE07.001'},
 {'layer': 'SRB2', 'product': 'CU_LE07.001'},
 {'layer': 'SRB3', 'product': 'CU_LE07.001'},
 {'layer': 'SRB4', 'product': 'CU_LE07.001'},
 {'layer': 'SRB5', 'product': 'CU_LE07.001'},
 {'layer': 'SRB7', 'product': 'CU_LE07.001'}]

In [79]:
## establish credendtial for the api
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd

## Enter Earth Data login Credentials
username = "lin_ncsu"#getpass.getpass('Earthdata Username:')
password = "skv6W]j24"#getpass.getpass('Earthdata Password:')

api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'  # Set the AρρEEARS API to a variable

token_response = r.post('{}login'.format(api), auth=(username, password)).json() # Insert API URL, call login service, provide credentials & return json
del username, password                                                           # Remove user and password information
token_response 

{'token_type': 'Bearer',
 'token': 'qsqr4jrhL-lVTHq4OFnMy5Wa0tCrScp7Zj_wOmYeRu4QnjIODKWYKUPY9MlMyWlUJXRiT_0KkU2NVqQnQm-U8w',
 'expiration': '2020-10-15T14:30:21Z'}

In [80]:
# look up the available projections 
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections
projs = {}
for p in projections:
    projs[p["Name"]] = p
list(projs.keys())



['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north',
 'laea_sphere_19']

In [81]:
## submit the orders by tiles
import os
import glob
import time

tile_list = glob.glob("D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles/*.shp")      # where the 75 tiles stored
short_name = [os.path.basename(x) for x in glob.glob("D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles/*.shp")]  # get the tile names

period1 = ["01-01-1999", "12-31-2004"]            # due to limits of APPEEARS' data process capability we need to download data by 2 periods
period2 = ["01-01-2005", "12-31-2010"]
period3 = ["01-01-2011", "12-31-2016"]
period4 = ["01-01-2017", "12-31-2018"]
periods = {"1" : period1, "2" : period2, "3":period3, "4":period4}

for p in ["1", "2", "3", "4"]:

    for i in range(0,74):

        nps = gpd.read_file(tile_list[i])
        nps = nps.to_crs("EPSG:4326")

        nps_gc = nps.to_json()
        nps_gc = json.loads(nps_gc)

        task_name = "p" + p + "_" + short_name[i][0:7]        # task name = period + tile name
        task_type = "area"                                    # we are using polygon instead of points to extract data
        proj = projs["geographic"]["Name"]                    # it MUST BE geographic projection make this api work
        outFormat = ["geotiff"]
        startDate = periods[p][0]
        endDate = periods[p][1]
        recurring = False

        task = {                                # build a json type of geo area to indicate ROI
            "task_type": task_type,             # area sample task
            "task_name": task_name,
            "params": {
                "dates": [
                {
                    "startDate": startDate,
                    "endDate": endDate
                }],
                "layers": prodLayer,
                "output": {
                    "format": {
                        "type": outFormat[0]    # geotiff format
                    },
                    "projection": proj
                },
                "geo": nps_gc
                }
        }


        token = token_response['token']                      # Save login token to a variable
        head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

        task_response = r.post("{}task".format(api), json = task, headers = head).json()
        print(task_response)

        task_id = task_response['task_id']                                               # Set task id from request submission
        status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
        print(status_response['status'])

        write_task_id = task_id + " : " + task_name + ":" + status_response['status']
        t = open(os.path.join("D:/Zekun/Landsat_ARD/Landsat_ARD/se_ard/", prods,"_task_id_by_tile.txt"), 'a')
        t.write(write_task_id +"\n")
        t.close()

        print('orders have been successfully submitted.')

{'task_id': 'ae94c133-aaf2-4668-8988-099541898526', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': 'd9d804c5-b532-4e94-9c2a-8f56704e75e4', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': 'e9790ee5-be00-4fd5-ba06-38f24dfcccbc', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': 'ac5d7f3c-c665-4c1e-897e-7b172ddc2f67', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': 'aabcca8b-5808-4943-a0cc-dadd4c47042e', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': 'dbef8c6e-0277-4b8e-b5e8-e91171653388', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': '3ea12482-430b-4185-8d8c-f8747af6e2c0', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': '2df2bdeb-5e32-488e-ae2a-4e8078c1d4c5', 'status': 'pending'}
pending
orders have been successfully submitted.
{'task_id': 'd9543cf1-03

KeyError: 'task_id'

In [89]:
## establish credendtial for the api
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd

## Enter Earth Data login Credentials
username = "ZekunLin"#getpass.getpass('Earthdata Username:')
password = "P6Fn8T47"#getpass.getpass('Earthdata Password:')

api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'  # Set the AρρEEARS API to a variable

token_response = r.post('{}login'.format(api), auth=(username, password)).json() # Insert API URL, call login service, provide credentials & return json
del username, password                                                           # Remove user and password information
token_response 

token = token_response['token']                                                     # retrieve the existed orders in the Appeears
response = r.get('https://lpdaacsvc.cr.usgs.gov/appeears/api/task', 
    headers={'Authorization': 'Bearer {0}'.format(token)})
task_response = response.json()

t = open("D:/Zekun/Landsat_ARD/Landsat_ARD/se_ard/LT5_task_id_by_tile.txt", "r")    # read the list of orders
task_id_list = t.read().split('\n')


for x in range(len(task_id_list)):
    submited_id = task_id_list[x][0:36]
    for y in range(len(task_response)):
        if task_response[y]['task_id'] == submited_id:
            if task_response[y]['status'] == 'done':
                download_this_name = task_response[y]['task_name']
                download_this_id = task_response[y]['task_id']
                bundle = r.get('{}bundle/{}'.format(api, download_this_id)).json() # Call API and return bundle contents for the task_id as json
                #print(bundle['files'])
                files = {}                                                         # Create empty dictionary
                for f in bundle['files']: files[f['file_id']] = f['file_name']    # Fill dictionary with file_id as keys and file_name as values

                destDir = os.path.join("D:\Zekun\Landsat_ARD\Landsat_ARD\se_ard", prods[0], download_this_name)
                if not os.path.exists(destDir) : os.makedirs(destDir)

                for f in files:
                    dl = r.get('{}bundle/{}/{}'.format(api, download_this_id, f), stream = True)                      # Get a stream to the bundle file
                    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])   # Parse the name from Content-Disposition header
                    filepath = os.path.join(destDir, filename)                                                        # Create output file path
                    with open(filepath, 'wb') as f:                                                                   # Write file to dest dir
                        for data in dl.iter_content(chunk_size=8192): f.write(data)
                print('Downloaded files can be found at: {}'.format(destDir))

    else: pass




83e69d90-8bd3-4314-b125-5cace2bfbbe9


' \nfor x in range(len(task_id_list)):\n    submited_id = task_id_list[x][0:36]\n    for y in range(len(task_response)):\n        if task_response[y][\'task_id\'] == submited_id:\n            if task_response[y][\'status\'] == \'done\':\n                download_this_name = task_response[y][\'task_name\']\n                download_this_id = task_response[y][\'task_id\']\n                bundle = r.get(\'{}bundle/{}\'.format(api, download_this_id)).json() # Call API and return bundle contents for the task_id as json\n                #print(bundle[\'files\'])\n                files = {}                                                         # Create empty dictionary\n                for f in bundle[\'files\']: files[f[\'file_id\']] = f[\'file_name\']    # Fill dictionary with file_id as keys and file_name as values\n\n                destDir = os.path.join("D:\\Zekun\\Landsat_ARD\\Landsat_ARD\\se_ard", prods[0], download_this_name)\n                if not os.path.exists(destDir) : os.ma

In [77]:
import glob
tile_list = glob.glob("D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles/*.shp")      # where the 75 tiles stored
for i in range(5, 56):
    print(tile_list[i])

D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h18_v17.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h19_v12.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h19_v13.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h19_v14.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h19_v15.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h19_v16.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h19_v17.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h20_v12.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h20_v13.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h20_v14.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h20_v15.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h20_v16.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h20_v17.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h21_v10.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles\h21_v11.shp
D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni

In [97]:

bundle = r.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json


files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values


destDir = os.path.join("D:\Zekun\Landsat_ARD\Landsat_ARD\se_ard\CU_LT05.001", task_name)
if not os.path.exists(destDir) : os.makedirs(destDir)

for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), stream=True)           # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  # Parse the name from Content-Disposition header 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir)) 

' bundle = r.get(\'{}bundle/{}\'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json\n\n\nfiles = {}                                                       # Create empty dictionary\nfor f in bundle[\'files\']: files[f[\'file_id\']] = f[\'file_name\']   # Fill dictionary with file_id as keys and file_name as values\n\n\ndestDir = os.path.join("D:\\Zekun\\Landsat_ARD\\Landsat_ARD\\se_ard\\CU_LT05.001", task_name)\nif not os.path.exists(destDir) : os.makedirs(destDir)\n\nfor f in files:\n    dl = r.get(\'{}bundle/{}/{}\'.format(api, task_id, f), stream=True)           # Get a stream to the bundle file\n    filename = os.path.basename(cgi.parse_header(dl.headers[\'Content-Disposition\'])[1][\'filename\'])  # Parse the name from Content-Disposition header \n    filepath = os.path.join(destDir, filename)                                                       # Create output file path\n    with open(filepath, \'wb\') as f:                                 